In [14]:
import numpy as np
from pulp import *
MEPS = 1.0e-6

In [15]:
def DEA_CCR(x, y, DMUs):
  m,n = x.shape
  s,n = y.shape

  res = []
  for o in range(n):
    prob = LpProblem('DMU_'+str(o), LpMaximize)
    v = [LpVariable('v'+str(i), lowBound=0,
                    cat='Continuous') for i in range(m)]
    u = [LpVariable('u'+str(i), lowBound=0,
                    cat='Continuous') for i in range(s)]
    
    prob += lpDot(u,y[:,o])

    prob += lpDot(v,x[:,o])==1, 'Normalize'+str(o)
    for j in range(n):
      prob += lpDot(u,y[:,j]) <= lpDot(v,x[:,j])

    prob.solve()
    vs = np.array([v[i].varValue for i in range(m)])
    us = np.array([u[i].varValue for i in range(s)])

    (eo,) = np.where(np.abs(np.dot(vs,x)-np.dot(us,y))<=MEPS)
    res.append([DMUs[o], value(prob.objective),
                set(eo), tuple(vs), tuple(us)])
  return res

In [16]:
x = np.array([[321.774, 1376.049, 64.716, 143.457,
    80.689, 64.395, 126.573, 23.969, 59.798, 35.94],
    [14682.739, 5320.232, 2676.52, 999.832, 3226.726,
      2361.317, 4780.944, 975.012, 1745.045, 1359.773]])
y = np.array([[46, 26, 27, 19, 17, 10, 12, 8, 8, 4],
  [37, 18, 23, 18, 10, 18, 8, 11, 12, 3],
  [38, 26, 17, 19, 15, 14, 21, 10, 8, 15]])

DMUs = ['アメリカ','中国','イギリス','ロシア','ドイツ','フランス',
     '日本','オーストラリア','イタリア','カナダ']

res = np.array(DEA_CCR(x,y, DMUs))
results = [[x[0],x[1], list(res[:,0][list(x[2])])] for x in res]
results = sorted(results, key=lambda x: x[1], reverse=True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sasa./miniforge3/envs/py38/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/80226cc11e74404c9811340940c185e4-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/80226cc11e74404c9811340940c185e4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 72 RHS
At line 84 BOUNDS
At line 85 ENDATA
Problem MODEL has 11 rows, 5 columns and 52 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10 (-1) rows, 3 (-2) columns and 30 (-22) elements
0  Obj -0 Dual inf 121 (3)
3  Obj 0.36978146
Optimal - objective value 0.36978146
After Postsolve, objective 0.36978146, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.3697814606 - 3 iterations time 0.002, Presolve 0.00
Opt

/var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/ipykernel_69130/717501174.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(DEA_CCR(x,y, DMUs))


In [17]:
results

[['オーストラリア', 1.000000002, ['カナダ', 'ロシア', 'オーストラリア']],
 ['ロシア', 1.0000000009999999, ['ロシア']],
 ['カナダ', 0.9999999978, ['カナダ', 'ロシア', 'オーストラリア']],
 ['イギリス', 0.9999999970000001, ['イギリス', 'オーストラリア']],
 ['フランス', 0.666693702, ['ロシア', 'オーストラリア']],
 ['イタリア', 0.581318664, ['ロシア', 'オーストラリア']],
 ['ドイツ', 0.567588247, ['イギリス', 'ロシア', 'オーストラリア']],
 ['日本', 0.4118923692, ['カナダ', 'ロシア', 'オーストラリア']],
 ['アメリカ', 0.3697814572, ['イギリス', 'オーストラリア']],
 ['中国', 0.257167573, ['ロシア']]]